<a href="https://colab.research.google.com/github/veridelisi/langchain/blob/main/06_vectorizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apply a Simple Bag-of-Words Approach


In [ ]:
# install spacy if you haven't done so already and download the small English model
!pip install -U spacy
!python -m spacy download en_core_web_sm

# install scikit learn
!pip install scikit-learn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
    '2 cups of flour',
    'replace the flour',
    'replace the keyboard in 2 minutes',
    'do you prefer Windows or Mac',
    'the Mac has the most noisy keyboard',
]
X = vectorizer.fit_transform(corpus)
X.todense()

# Classification

The simplified brown corpus is available in the data folder of this github repository


In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/alexisperrier/intro2nlp/master/data/brown_corpus_extract_humor_science_fiction.csv"
df = pd.read_csv(url)
df.shape

print(df.topic.value_counts())

humor              1052
science_fiction     948
Name: topic, dtype: int64


In [ ]:
df.head()

,topic,text
0,humor,"They were always leaping to light cigarettes, ..."
1,science_fiction,"No, the fish could eat their bodies for all th..."
2,science_fiction,"Fighting pleasure itself, he begged B'dikkat t..."
3,science_fiction,that is all I can tell you about them.
4,humor,"That's one thing I've never done , she said p..."


## Load spacy


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

Define a function that

- removes stopwords
- removes punctuation signs
- lemmatizes the text


In [ ]:
def preprocess(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not (token.is_stop or token.is_punct)]
    return ' '.join(tokens)


In [ ]:
text = '''These are the good times; Leave your cares behind'''
print(preprocess(text))

good time leave care


Add a couple of punctuation signs with multi charaters as stopwords to the ```nlp``` spacy model

In [ ]:
nlp.Defaults.stop_words.add("`,")
nlp.Defaults.stop_words.add("``")

Let's apply our preprocess function to the simplified Brown Corpus

In [ ]:
df['processed_text'] = df.text.apply(lambda txt : preprocess(txt))

short text with just a few tokens won't have enough information for the classification model that we want to train. Let's add a count of the number of tokens for each text and filter out the text with less than N tokens.

In [ ]:
df['token_count'] = df.processed_text.apply(lambda txt : len(txt.split())  )

In [ ]:
df.token_count.describe()

count    2000.000000
mean        6.750500
std         5.484641
min         0.000000
25%         3.000000
50%         5.000000
75%         9.000000
max        46.000000
Name: token_count, dtype: float64

In [ ]:
df.token_count.hist(bins = 100)

Let's remove texts that have less than 4 tokens

In [ ]:
df = df[df.token_count > 4]
df.reset_index(inplace = True, drop = True)

In [ ]:
df[df.token_count > 4].topic.value_counts()

## Vectorize the Data

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(df.processed_text)

X is now a sparse matrix of 1161 rows by 4732 columns

In [ ]:
X.shape

(1161, 4732)

Transform the topic from string to integer

In [ ]:
df.loc[df.topic == 'humor', 'topic' ] = 0
df.loc[df.topic == 'science_fiction', 'topic' ] = 1
df.topic = df.topic.astype(int)
# define the target variable
y = df.topic

In [ ]:
y

0       0
1       1
2       1
3       0
4       1
       ..
1156    0
1157    0
1158    1
1159    1
1160    1
Name: topic, Length: 1161, dtype: int64

In [ ]:
X

<1161x4732 sparse matrix of type '<class 'numpy.int64'>'
	with 11296 stored elements in Compressed Sparse Row format>

## Train a classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# 1. Declare the model
clf = MultinomialNB()

# 2. Train the model
clf.fit(X, y)

# 3. Make predictions
yhat = clf.predict(X)

# 4. score
print("Accuracy: ",accuracy_score(y, yhat))

Accuracy:  0.9879414298018949


To really train a classifier, we need to split the data in two parts: a training subset on which we train the model and a test set on which we evaluate the model.

The test set simulates data that the model has not seen during its training and gives us a way to measure how the model extrapolates on unseen data.

We need to split the data after we've vectorized the text, otherwise sont tokens may be present in the test set but not in the training set.

To split the data into a train and a test set we use scikit's [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function.



In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)

(777, 4732)
(384, 4732)


Let's train the same model, this time on the train set and evaluate it on the test set



In [ ]:
# 1. Declare the model
clf = MultinomialNB()

# 2. Train the model
clf.fit(X_train, y_train)

# 3. Make predictions  on test set
yhat = clf.predict(X_test)

# 4. score
print("Accuracy: ",accuracy_score(y_test, yhat))

Accuracy:  0.8229166666666666


The accuracy is down to 77.4% which makes more sense than the outstanding 98% we previously obtained.

# tf-idf

Let's now compare the model performance when we use a tf-idf vectorization approach instead of a simple tf / count vectorizer.

The code is similar and we use scikit's default parameter for [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer()
X = cv.fit_transform(df.processed_text)
y = df.topic

# split test train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# train and evaluate the model
# 1. Declare the model
clf = MultinomialNB()

# 2. Train the model
clf.fit(X_train, y_train)

# 3. Make predictions  on test set
yhat = clf.predict(X_test)

# 4. score
print("Accuracy: ",accuracy_score(y_test, yhat))

Accuracy:  0.7890625


In this particular case, not much difference between the 2 vectorizers.